# __ÁRVORES DE DECISÃO__

***
## _Prompts e Respostas_

` Prompt 1: Necessito que crie um algoritmo sem o uso de bibliotecas como o 'scikit-learn' para o método de aprendizado supervisionado Árvores de Decisão que lide com valores numéricos e categóricos utilizando Gini para verificar a pureza dos nós na linguagem Python (Jupyter Notebook)`

>Claro, vou lhe fornecer um exemplo simples de como implementar um algoritmo de Árvore de Decisão utilizando o critério de Gini para medir a pureza dos nós, sem depender da biblioteca scikit-learn. Aqui está um exemplo básico em Python:

In [89]:
import numpy as np

class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index  # Índice do recurso usado para dividir os dados
        self.threshold = threshold  # Valor de divisão
        self.left = left  # Subárvore à esquerda
        self.right = right  # Subárvore à direita
        self.value = value  # Valor da classe para folhas (somente se for uma folha)

class DecisionTreeClassifier:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.num_classes = len(set(y))
        self.num_features = X.shape[1]
        self.tree = self._grow_tree(X, y)

    def _gini(self, y):
        classes = set(y)
        n_instances = len(y)
        gini = 1.0
        for cls in classes:
            proportion = np.sum(y == cls) / n_instances
            gini -= proportion ** 2
        return gini

    def _best_split(self, X, y):
        best_gini = float('inf')
        best_criteria = None  # Critérios para divisão: (feature_index, threshold)
        
        for feature_index in range(self.num_features):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                left_indices = np.where(X[:, feature_index] <= threshold)[0]
                right_indices = np.where(X[:, feature_index] > threshold)[0]
                
                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue
                
                gini = self._gini_split(y[left_indices], y[right_indices])
                
                if gini < best_gini:
                    best_gini = gini
                    best_criteria = (feature_index, threshold)
        
        return best_criteria

    def _gini_split(self, left_y, right_y):
        p_left = len(left_y) / (len(left_y) + len(right_y))
        gini_left = self._gini(left_y)
        gini_right = self._gini(right_y)
        gini = p_left * gini_left + (1 - p_left) * gini_right
        return gini

    def _grow_tree(self, X, y, depth=0):
        num_samples_per_class = [np.sum(y == i) for i in range(self.num_classes)]
        predicted_class = np.argmax(num_samples_per_class)
        node = Node(value=predicted_class)
        
        if depth < self.max_depth:
            criteria = self._best_split(X, y)
            if criteria:
                feature_index, threshold = criteria
                left_indices = np.where(X[:, feature_index] <= threshold)[0]
                right_indices = np.where(X[:, feature_index] > threshold)[0]
                left_subtree = self._grow_tree(X[left_indices], y[left_indices], depth + 1)
                right_subtree = self._grow_tree(X[right_indices], y[right_indices], depth + 1)
                node = Node(feature_index=feature_index, threshold=threshold, left=left_subtree, right=right_subtree)
        return node

    def predict(self, X):
        return [self._predict_tree(x, self.tree) for x in X]

    def _predict_tree(self, x, tree):
        if tree.value is not None:
            return tree.value
        feature_value = x[tree.feature_index]
        if feature_value <= tree.threshold:
            return self._predict_tree(x, tree.left)
        else:
            return self._predict_tree(x, tree.right)


`Prompt 2: Obrigado! Agora quero que você utilize o dataset "Penguins" para testar esse algoritmo, sem o uso de bibliotecas como o skitlearn`

>Claro! Vou utilizar o conjunto de dados "Penguins" para testar o algoritmo de Árvore de Decisão que acabamos de implementar. Primeiro, vamos carregar e visualizar os dados:

In [90]:
import pandas as pd

# Carregar o conjunto de dados penguins
penguins_data = pd.read_csv("https://raw.githubusercontent.com/dataprofessor/data/master/penguins_cleaned.csv")

# Visualizar as primeiras linhas do conjunto de dados
print(penguins_data.head())


  species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0  Adelie  Torgersen            39.1           18.7                181   
1  Adelie  Torgersen            39.5           17.4                186   
2  Adelie  Torgersen            40.3           18.0                195   
3  Adelie  Torgersen            36.7           19.3                193   
4  Adelie  Torgersen            39.3           20.6                190   

   body_mass_g     sex  
0         3750    male  
1         3800  female  
2         3250  female  
3         3450  female  
4         3650    male  


>Agora, vamos pré-processar os dados para que possam ser usados no nosso algoritmo de Árvore de Decisão:

In [91]:
import pandas as pd
import numpy as np

# Remover linhas com valores nulos
penguins_data.dropna(inplace=True)

# Converter variáveis categóricas em numéricas usando one-hot encoding manualmente
penguins_data = pd.concat([penguins_data, pd.get_dummies(penguins_data['species'], prefix='species')], axis=1)
penguins_data = pd.concat([penguins_data, pd.get_dummies(penguins_data['island'], prefix='island')], axis=1)
penguins_data = pd.concat([penguins_data, pd.get_dummies(penguins_data['sex'], prefix='sex')], axis=1)

# Remover as colunas originais categóricas
penguins_data.drop(['species', 'island', 'sex'], axis=1, inplace=True)

# Separar os dados em features (X) e target (y)
X = penguins_data.drop('species_Chinstrap', axis=1).values
y = penguins_data['species_Chinstrap'].values

# Dividir os dados manualmente em conjunto de treinamento e conjunto de teste
def manual_train_test_split(X, y, test_size=0.2, random_state=None):
    if random_state:
        np.random.seed(random_state)
        
    num_samples = len(X)
    indices = np.random.permutation(num_samples)
    test_size = int(test_size * num_samples)
    
    X_train = X[indices[:-test_size]]
    X_test = X[indices[-test_size:]]
    y_train = y[indices[:-test_size]]
    y_test = y[indices[-test_size:]]
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = manual_train_test_split(X, y, test_size=0.2, random_state=42)


`Prompt 4: Agora como posso fazer para apresentar as métricas?`
>Para apresentar as métricas do modelo, como a acurácia, você pode implementar essas métricas manualmente. Aqui está um exemplo de como você pode calcular a acurácia manualmente:

In [92]:
def accuracy_score(y_true, y_pred):
    correct = 0
    total = len(y_true)
    for true_label, pred_label in zip(y_true, y_pred):
        if true_label == pred_label:
            correct += 1
    return correct / total

# Fazer previsões
predictions = tree.predict(X_test)

# Calcular a acurácia
accuracy = accuracy_score(y_test, predictions)
print("Acurácia do modelo:", accuracy)

Acurácia do modelo: 1.0


` Prompt 5: A acurácia do modelo está resultando em 1, isso não significa overfitting? como eu resolvo?`

>Uma acurácia de 1.0 (ou 100%) nos dados de teste pode indicar overfitting, especialmente se o modelo não estiver sendo avaliado em um conjunto de validação. Overfitting ocorre quando o modelo se ajusta demais aos dados de treinamento e não generaliza bem para dados novos.

>Para resolver o overfitting, você pode tentar algumas abordagens:

>Reduzir a complexidade do modelo: Reduza a profundidade da árvore de decisão ou aplique técnicas de poda para simplificar a estrutura da árvore.

>Regularização: Introduza parâmetros de regularização, como limitar o número máximo de folhas em uma árvore ou introduzir penalidades nos pesos dos nós.

>Cross-validation: Use técnicas de validação cruzada para avaliar o desempenho do modelo em diferentes subconjuntos dos dados e encontrar hiperparâmetros que generalizam bem.

>Aumentar o tamanho do conjunto de teste: Se o tamanho do conjunto de teste for pequeno, é possível que a acurácia seja superestimada. Aumente o tamanho do conjunto de teste para uma estimativa mais confiável do desempenho do modelo.

>Vamos implementar a validação cruzada para avaliar melhor o desempenho do modelo e evitar o overfitting. Aqui está um exemplo de como você pode fazer isso:

In [87]:
import numpy as np

# Definir uma função para dividir os dados em K folds
def k_fold_split(X, y, num_folds=5, random_state=None):
    if random_state:
        np.random.seed(random_state)
    
    num_samples = len(X)
    fold_size = num_samples // num_folds
    indices = np.random.permutation(num_samples)
    
    for i in range(num_folds):
        start = i * fold_size
        end = (i + 1) * fold_size if i < num_folds - 1 else num_samples
        
        test_indices = indices[start:end]
        train_indices = np.concatenate((indices[:start], indices[end:]))
        
        yield X[train_indices], X[test_indices], y[train_indices], y[test_indices]

# Definir uma função para calcular a acurácia
def accuracy_score(y_true, y_pred):
    correct = np.sum(y_true == y_pred)
    total = len(y_true)
    return correct / total

# Realizar validação cruzada
num_folds = 5
accuracies = []

for X_train_fold, X_test_fold, y_train_fold, y_test_fold in k_fold_split(X_train, y_train, num_folds=num_folds, random_state=42):
    # Treinar o modelo em um fold
    tree_fold = DecisionTreeClassifier(max_depth=3)
    tree_fold.fit(X_train_fold, y_train_fold)
    
    # Fazer previsões no conjunto de teste do fold atual
    predictions_fold = tree_fold.predict(X_test_fold)
    
    # Calcular a acurácia para este fold
    accuracy_fold = accuracy_score(y_test_fold, predictions_fold)
    accuracies.append(accuracy_fold)

# Calcular a acurácia média da validação cruzada
mean_accuracy = np.mean(accuracies)
print("Acurácia média da validação cruzada:", mean_accuracy)

Acurácia média da validação cruzada: 1.0


>Essencialmente, esta implementação divide os dados em num_folds partes, treina o modelo em cada parte (fold) e avalia o desempenho do modelo em um conjunto de teste separado para cada fold. Finalmente, calcula a média das acurácias obtidas em cada fold para obter a acurácia média da validação cruzada.

***
## _Aplicando o Código_

***
## _Análise e Considerações_